In [ ]:
# This instruction is needed for installing the graphics library
!pip3 install ColabTurtle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ColabTurtle: filename=ColabTurtle-2.1.0-py3-none-any.whl size=7657 sha256=3b9eb8dcc1eabde23eac410982392a7b7af600fb7d6608a3fca1cf4978c41b76
  Stored in directory: /root/.cache/pip/wheels/0d/ab/65/cc4478508751448dfb4ecb20a6533082855c227dfce8c13902
Successfully built ColabTurtle


In [ ]:
#Dibujamos el tablero y la colocación de las fichas mediante circulos sobre el tablero:
def draw_square(side): 
  for i in range(0,4): 
    forward(side) 
    right(90) 

def draw_board(cell_side):
  bgcolor("blue")
  color("white")
  for i in range (0,window_width,cell_side):
    for j in range (0,window_height+cell_side,cell_side):
      penup()
      goto(i,j)
      pendown()
      draw_square(cell_side)

def place_piece(col,row,piece_color,cell_side): 
  penup()
  goto(col*cell_side+25,int((row)*cell_side + cell_side/2))
  pendown()
  color(piece_color)
  width(10)
  
  for i in range(0,18): 
    forward(10) 
    right(20)

#Creamos una matriz nula, donde los usuarios podrán ir colocando sus fichas mediante el número 1 y 2.  
def print_matriz(matriz):
  for fila in range(len(matriz[0])):
    for columna in range(len(matriz)):
      print(matriz[columna][fila], end=" ")
    print()

#Miramos si en la columna donde el usuario quiere poner su ficha está libre. 
def primera_pos_libre(matriz, columna):
  ultimo_0 = -1
  for i in range(len(matriz[columna])):
    if matriz[columna][i] == 0:
      ultimo_0 = i
  return ultimo_0

# Preguntamos el color que quiere el jugador y nos aseguramos que ponga el amarillo o el rojo
def color_del_jugador(color_jug):
  while color_jug != "amarillo" and color_jug != "rojo":
    color_jug = input("Color del jugador 1, amarillo o rojo. ")
    if color_jug != "amarillo" and color_jug != "rojo":
      print("Tienes que elegir el rojo o el amarillo, no hay más opciones")
  return color_jug

# Pedimos al usuario que columna quiere y miramos que esta esté dentro del tablero y que no dé error
def pedir_columna(col):
  while col == -1:
    try:
      col = int(input("En qué columna pones la ficha (entre 0 y 6)? "))
    except:
      print("Tienes que poner un número.")
  while col < 0 or col > 6:
    print("Esta columna no existe.")
    col = False
    while col == False:
      try:
        col = int(input("Elige una columna entre 0 y 6. "))
      except:
        print("Pon un número.")
  return col

# Miramos las situaciones en que un jugadoor puede ganar
def posicion_ganador(colums, fils, matrix, posicion):

# Vemos las posiciones horizontales
  for x in range(colums):
    for y in range(fils-3):
      if matrix[x][y] == posicion and matrix[x][y+1] == posicion and matrix[x][y+2] == posicion and matrix[x][y+3] == posicion:
        return True
        
# Vemos las posiciones verticales
  for x in range(colums-3):
    for y in range(fils):
      if matrix[x][y] == posicion and matrix[x+1][y] == posicion and matrix[x+2][y] == posicion and matrix[x+3][y] == posicion:
        return True

# Vemos las posiciones diagonales descendientes
  for x in range(colums-3):
    for y in range(fils-3):
      if matrix[x][y]== posicion and matrix[x+1][y+1] == posicion and matrix[x+2][y+2] == posicion and matrix[x+3][y+3] == posicion:
        return True

# Y vemos la posiciones diagonales ascendientes
  for x in range(3, colums):
    for y in range(fils-3):
      if matrix[x][y] == posicion and matrix [x-1][y+1] == posicion and matrix[x-2][y+2] == posicion and matrix[x-3][y+3] == posicion:
        return True

# Miramos si los jugadores quedan empate
def empate(matriz):
  if matriz[0][0] != 0 and matriz[1][0] != 0 and matriz[2][0] != 0 and matriz[3][0] != 0 and  matriz[4][0] !=0 and matriz[5][0] !=0 and matriz[6][0] !=0 :
    return True

# Inteligencia aritficial mira si hay 3 fichas iguales del contrincante en posición vertical y horizontal
def ficha_IA_iguales(matrix, colums, fils):
  for x in range(colums):
    for y in range(fils-2):
      if matrix[x][y] == 1 and matrix[x][y+1] == 1 and matrix[x][y+2] == 1:
        colum = matrix[x][y+2]
        return colum

  for x in range(colums-2):
    for y in range(fils):
      if matrix[x][y] == 1 and matrix[x+1][y] == 1 and matrix[x+2][y] == 1:
        colum = matrix[x+2][y]
        return colum

# La IA detecta si está a punto de ganar en posición horizontal o vertical, por tal poner la ficha al lado o arriba y ganar 
def ficha_IA_ganar(matrix, colums, fils):
  for x in range(colums):
    for y in range(fils-2):
      if matrix[x][y] == 2 and matrix[x][y+1] == 2 and matrix[x][y+2] == 2:
        colum = matrix[x][y+2]
        return colum

  for x in range(colums-2):
    for y in range(fils):
      if matrix[x][y] == 2 and matrix[x+1][y] == 2 and matrix[x+2][y] == 2:
        colum = matrix[x+2][y]
        return colum
  

In [ ]:
import random
from ColabTurtle.Turtle import *
window_width = 700
window_height = 600
cursor_speed = 10

initializeTurtle(cursor_speed,(window_width, window_height)) 

cell_side = 100
draw_board(cell_side)
 

filas = 6
columnas = 7
ganador = False
posiciones_libres = True 
modo_juego = False
col = -1
color_jug = -1

# Generamos la matriz para que el programa lea las casillas. Más adelante para observar la matriz se tendrá que eliminar el comentario print_matriz(matriz)
matriz = []
for i in range(columnas):
  matriz.append([0]*filas)
#print_matriz(matriz)

print("""Hay dos modos de juego:
1. Jugador vs jugador
2. Jugador vs IA (donde el jugador es el 1 y la el ordenador es el 2)""")

#Preguntamos el modo de juego y lo metemos en una lista junto al color que ha elegido el jugador 1
while modo_juego == False:
  try:
    modo_juego = int(input("Qué modo de juego quieres (1 o 2)? "))
  except:
    print("Pon lo que dice la pregunta.")

tipo_de_juego = [modo_juego, color_del_jugador(color_jug)]

#Vemos quien empieza aleatoriamente
primer_jugador = random.randint(1, 2)

# mientras no haya ganador
while ganador == False:  
  
  # le toca a uno de los dos alternativamente
  if primer_jugador == 2:
    primer_jugador = 1
  else:
    primer_jugador = 2
  print("Le toca al jugador", primer_jugador)

  if tipo_de_juego[0] == 1:
    # Llamamos a la función para preguntar en que columna quieren poner la ficha
    colum = pedir_columna(col)

    # Comprobamos que la columna donde quieren poner la ficha esté vacía. Si esta llena volvemos a preguntar.  
    pos_libre = primera_pos_libre(matriz, colum)
    if pos_libre == -1:
      while pos_libre == -1:
        try:
          colum = int(input("Esta columna esta llena, pon otra. "))
          pos_libre = primera_pos_libre(matriz, colum)
        except:
          print("Tienes que poner un número")
      while colum < 0 or colum > 6:
        print("Esa columna está fuera del rango.")
        colum = -1
        while colum == -1:
          try:
            colum = int(input("Tiene que ser una columna entre el 0 y el 6. "))
          except:
            print("Tiene que ser un número.")

  # ponemos la ficha, si el jugador 1 elige rojo:
    if tipo_de_juego[1] == "rojo":
      if primer_jugador == 1:
        matriz[colum][pos_libre] = 1
        pieza = primer_jugador
        #print_matriz(matriz)
        place_piece(colum,pos_libre,"red",cell_side)

      else:
        matriz[colum][pos_libre] = 2
        pieza = primer_jugador
        #print_matriz(matriz)
        place_piece(colum,pos_libre,"yellow",cell_side)
        
    # Si el jugador 1 elige amarillo
    elif tipo_de_juego[1] == "amarillo":
      if primer_jugador == 1:
        matriz[colum][pos_libre] = 1
        pieza = primer_jugador
        #print_matriz(matriz)
        place_piece(colum,pos_libre,"yellow",cell_side)

      else:
        matriz[colum][pos_libre] = 2
        pieza = primer_jugador
        #print_matriz(matriz)
        place_piece(colum,pos_libre,"red",cell_side)

    # Llamamos a la función para ver el ganador y si hay sale del bucle.
    if posicion_ganador(columnas, filas, matriz, pieza):
      print("El jugador", primer_jugador, "ha ganado. ¡Enhorabuena!")
      ganador = True
    
    #Llamamos a la función y vemos si los jugadores han empatado. Si es así sale del bucle.
    elif empate(matriz):
      print("Habéis empatado")
      ganador = True
    
  elif tipo_de_juego[0] == 2:

    if primer_jugador == 1:
      col = -1
      colum = pedir_columna(col)
    
    elif primer_jugador == 2:
     colum = random.randint(0,6)
      

    ### Se han mantenido los códigos siguientes en forma de comentario por tal de que no interfieran en el funcionamiento de "vs. máquina estúpida". ###
    # Se llaman a las diferenctes funciones para que la IA ponga la ficha donde debe, si no hay ningún caso la pone aleatoriamente
    #elif primer_jugador == 2:
      #if ficha_IA_iguales(matriz, columnas, filas):       
        #pos_libre = primera_pos_libre(matriz, colum)
      #elif ficha_IA_ganar(matriz, columnas, filas):
        #pos_libre = primera_pos_libre(matriz, colum)
        #ganador = True
      #else:
        #colum = random.randint(0,6)
      
        
    # Miramos la última posición para poner la ficha
    pos_libre = primera_pos_libre(matriz, colum)
    if pos_libre == -1:
      while pos_libre == -1:
        try:
          colum = int(input("Esta columna esta llena, pon otra. "))
          pos_libre = primera_pos_libre(matriz, colum)
        except:
          print("Tienes que poner un número")
      while colum < 0 or colum > 6:
        print("Esa columna está fuera del rango.")
        colum = -1
        while colum == -1:
          try:
            colum = int(input("Tiene que ser una columna entre el 0 y el 6. "))
          except:
            print("Tiene que ser un número.")

    # Repetimos lo hecho antes para colocar las fichas según el color elegido por el usuario
    if tipo_de_juego[1] == "rojo":

      if primer_jugador == 1:
        matriz[colum][pos_libre] = 1
        pieza = primer_jugador
        #print_matriz(matriz)
        place_piece(colum,pos_libre,"red",cell_side)

      else:
        matriz[colum][pos_libre] = 2
        pieza = primer_jugador
        #print_matriz(matriz)
        place_piece(colum,pos_libre,"yellow",cell_side)
        
    elif tipo_de_juego[1] == "amarillo":    
      if primer_jugador == 1:
        matriz[colum][pos_libre] = 1
        pieza = primer_jugador
        #print_matriz(matriz)
        place_piece(colum,pos_libre,"yellow",cell_side)

      else:
        matriz[colum][pos_libre] = 2
        pieza = primer_jugador
        #print_matriz(matriz)
        place_piece(colum,pos_libre,"red",cell_side)
    
    # Miramos si hay algún jugador que ha ganado o empatado, y si es el caso se sale del bucle:
    if posicion_ganador(columnas, filas, matriz, pieza):
      print("El jugador", primer_jugador, "ha ganado. ¡Enhorabuena!")
      ganador = True
    
    elif empate(matriz):
      print("Habéis empatado.")
      ganador = True
  else:
    print("Sólo hay dos modos de juego. Elige entre el 1 y el 2")

Hay dos modos de juego:
1. Jugador vs jugador
2. Jugador vs IA (donde el jugador es el 1 y la el ordenador es el 2)
Qué modo de juego quieres (1 o 2)? 2
Color del jugador 1, amarillo o rojo. rojo
Le toca al jugador 1
En qué columna pones la ficha (entre 0 y 6)? 0
Le toca al jugador 2
Le toca al jugador 1
En qué columna pones la ficha (entre 0 y 6)? 7
Esta columna no existe.
Elige una columna entre 0 y 6. 5
Le toca al jugador 2
Le toca al jugador 1
En qué columna pones la ficha (entre 0 y 6)? 3
Le toca al jugador 2
Le toca al jugador 1
En qué columna pones la ficha (entre 0 y 6)? 2
Le toca al jugador 2
Le toca al jugador 1
En qué columna pones la ficha (entre 0 y 6)? 1
Le toca al jugador 2
Le toca al jugador 1
